# The Diet Problem
Source: https://notebook.community/Pyomo/PyomoGallery/diet/DietProblem

## Summary

The goal of the Diet Problem is to select foods that satisfy daily nutritional requirements at minimum cost. This problem can be formulated as a linear program, for which constraints limit the number of calories and the amount of vitamins, minerals, fats, sodium, and cholesterol in the diet. Danzig (1990) notes that the diet problem was motivated by the US Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet.

## Problem Statement

The Diet Problem can be formulated mathematically as a linear programming problem using the following model.  

### Sets

 $F$ = set of foods  
 $N$ = set of nutrients

### Parameters

 $c_i$ = cost per serving of food $i$, $\forall i \in F$  
 $a_{ij}$ = amount of nutrient $j$ in food $i$, $\forall i \in F, \forall j \in N$  
 $Nmin_j$ = minimum level of nutrient $j$, $\forall j \in N$  
 $Nmax_j$ = maximum level of nutrient $j$, $\forall j \in N$  
 $V_i$ = the volume per serving of food $i$, $\forall i \in F$  
 $Vmax$ = maximum volume of food consumed
 
### Variables
 $x_i$ = number of servings of food $i$ to consume

### Objective

Minimize the total cost of the food  
 $\min \sum_{i \in F} c_i x_i$

### Constraints

Limit nutrient consumption for each nutrient $j \in N$.  
 $Nmin_j \leq \sum_{i \in F} a_{ij} x_i \leq Nmax_j$, $\forall j \in N$

Limit the volume of food consumed  
 $\sum_{i \in F} V_i x_i \leq Vmax$
 
Consumption lower bound  
 $x_i \geq 0$, $\forall i \in F$

## Pyomo Formulation

We begin by importing the Pyomo package and creating a model object:

In [1]:
from pyomo.environ import *
infinity = float('inf')

model = AbstractModel(name="Diet")

The sets $F$ and $N$ are declared abstractly using the `Set` component:

In [2]:
# Foods
model.F = Set()
# Nutrients
model.N = Set()

Similarly, the model parameters are defined abstractly using the `Param` component:

In [3]:
# Cost of each food
model.c    = Param(model.F, within=PositiveReals)
# Amount of nutrient in each food
model.a    = Param(model.F, model.N, within=NonNegativeReals)
# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
# Volume per serving of food
model.V    = Param(model.F, within=PositiveReals)
# Maximum volume of food consumed
model.Vmax = Param(within=PositiveReals)

The `within` option is used in these parameter declarations to define expected properties of the parameters.  This information is used to perform error checks on the data that is used to initialize the parameter components.

The `Var` component is used to define the decision variables:

In [4]:
# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)

The `within` option is used to restrict the domain of the decision variables to the non-negative reals. This eliminates the need for explicit bound constraints for variables.

The `Objective` component is used to define the cost objective.  This component uses a rule function to construct the objective expression:

In [5]:
# Minimize the cost of food that is consumed
def cost_rule(model):
    return sum(model.c[i]*model.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

Similarly, rule functions are used to define constraint expressions in the `Constraint` component:

In [6]:
# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j):
    value = sum(model.a[i,j]*model.x[i] for i in model.F)
    return inequality(model.Nmin[j], value, model.Nmax[j])
model.nutrient_limit = Constraint(model.N, rule=nutrient_rule)

# Limit the volume of food consumed
def volume_rule(model):
    return sum(model.V[i]*model.x[i] for i in model.F) <= model.Vmax
model.volume = Constraint(rule=volume_rule)

Putting these declarations all together gives the model.

## Model Data

Since this is an abstract Pyomo model, the set and parameter values need to be provided to initialize the model. The data is in the command file diet.dat

Set data is defined with the `set` command, and parameter data is defined with the `param` command.

This data set considers the problem of designing a daily diet with only food from a fast food chain.

## Solution

Pyomo includes a `pyomo` command that automates the construction and optimization of models.  The GLPK solver can be used in this simple example:

In [7]:
instance = model.create_instance("diet.dat")
solver = SolverFactory('glpk')
result = solver.solve(instance, tee = False)
instance.display()

Model Diet

  Variables:
    x : Size=9, Index=F
        Key              : Lower : Value : Upper : Fixed : Stale : Domain
            Cheeseburger :     0 :   4.0 :  None : False : False : NonNegativeIntegers
        Chicken Sandwich :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           Fish Sandwich :     0 :   1.0 :  None : False : False : NonNegativeIntegers
                   Fries :     0 :   5.0 :  None : False : False : NonNegativeIntegers
            Ham Sandwich :     0 :   0.0 :  None : False : False : NonNegativeIntegers
               Hamburger :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             Lowfat Milk :     0 :   4.0 :  None : False : False : NonNegativeIntegers
            Orange Juice :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         Sausage Biscuit :     0 :   0.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
      

## Interpretation of solution
This solution shows that for $15.05 per day, a person can get by with 4 cheeseburgers, 5 fries, 1 fish sandwich and 4 milks.

## References

* G.B. Dantzig. The Diet Problem, Interfaces 20(4), 1990, 43-47